##  Predicting Baseball Hall of Fame Induction

###  Data Analysis

####  Eryk Wdowiak and Ken Hoffman

data from Lahman Baseball Database

# Catchers

## Modeling

In [1]:
import warnings
# warnings.filterwarnings('ignore',category=RuntimeWarning)

import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score

from sklearn import metrics 


from matplotlib import pyplot as plt
pd.set_option('display.max_columns', 500)


from imblearn.over_sampling import SMOTE

# import matplotlib.pyplot as plt
# %matplotlib inline

In [2]:
#import dataframe
df = pd.read_pickle('catchers-df_2020-08-19a.p')

In [3]:
df['AVG'] = df['H']/df['AB']
df['OBP'] = (df['H'] + df['BB'] + df['HBP'])/(df['AB']+df['BB'] + df['HBP'] + df['SH'])

In [4]:
df = df.drop(['bCS', 'SH', 'HBP', 'bCS_ps', 'SH_ps', 'HBP_ps', 'teamID'], axis=1)

In [5]:
df.head()

,playerID,bG,AB,R,H,2B,3B,HR,RBI,bSB,BB,SO,bG_ps,AB_ps,R_ps,H_ps,2B_ps,3B_ps,HR_ps,RBI_ps,bSB_ps,BB_ps,SO_ps,fG,PO,A,E,DP,fSB,fCS,fG_ps,PO_ps,A_ps,E_ps,DP_ps,fSB_ps,fCS_ps,nu_sns,lst_sn,induct,position,AVG,OBP
17,alomasa02,1377,4530,520,1236,249,10,112,588.0,25.0,212,499.0,49.0,173.0,17.0,37.0,9.0,1.0,5.0,28.0,0.0,7.0,26.0,1325,7667,464,86.0,72,758.0,328.0,49.0,337.0,15.0,7.0,1.0,0.0,0.0,20,2007,0,catcher,0.272848,0.308218
35,ashbyal01,1370,4123,397,1010,183,13,90,513.0,7.0,461,622.0,11.0,40.0,3.0,5.0,1.0,0.0,2.0,5.0,0.0,4.0,1.0,1305,7096,685,114.0,75,1112.0,445.0,11.0,94.0,5.0,0.0,0.0,0.0,0.0,17,1989,0,catcher,0.244967,0.318710
37,astrojo01,544,1579,163,401,51,10,13,156.0,6.0,177,124.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,511,2057,233,31.0,46,46.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,1956,0,catcher,0.253958,0.330726
40,ausmubr01,1971,6279,718,1579,270,34,80,607.0,102.0,634,1034.0,35.0,106.0,12.0,26.0,5.0,0.0,3.0,7.0,1.0,9.0,29.0,1960,12856,960,80.0,133,894.0,480.0,35.0,245.0,18.0,0.0,2.0,0.0,0.0,18,2010,0,catcher,0.251473,0.323046
43,azcuejo01,909,2828,201,712,94,9,50,304.0,5.0,207,344.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,868,5329,452,44.0,67,243.0,201.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,11,1972,0,catcher,0.251768,0.305583


In [6]:
df['induct'].value_counts()

0    227
1     10
Name: induct, dtype: int64

In [7]:
df.shape

(237, 43)

In [8]:
# Create matrix of features
X = df.drop(['induct', 'position', 'playerID'], axis = 1) 

# Create target variable
y = df['induct']

feature_cols = X.columns

In [9]:
X.shape

(237, 40)

In [10]:
# Use x and y variables to split the training data into train and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,test_size=0.20)

## Logistic Regression

### W/ Class Imbalance

In [11]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(solver='liblinear')

lr_clf.fit(X_train, y_train)

y_pred_test = lr_clf.predict(X_test)
y_pred_train = lr_clf.predict(X_train)


# checking accuracy
print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_test))


# checking accuracy
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_test))
print('Train F1 Score: ', metrics.f1_score(y_train, y_pred_train))

Test Accuracy score:  0.9791666666666666
Test F1 score:  0.6666666666666666
Train F1 Score:  1.0


## Decision Tree

In [12]:
clf = DecisionTreeClassifier()


In [13]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#predict the training set
y_pred_train = clf.predict(X_train)

#Predict the response for test dataset
y_pred_test = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Testing Accuracy Score:",metrics.accuracy_score(y_test, y_pred_test))
print("Testing F1 Score:",metrics.f1_score(y_test, y_pred_test))

Testing Accuracy Score: 0.9583333333333334
Testing F1 Score: 0.6666666666666666


### GridsearchCV

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
#create a dictionary of all the parameters you want to tune
param_grid = { 
    'max_features': [.25, .5, .75],
    'max_depth' : [5, 10, 15],
    'criterion' : ['gini','entropy']}

In [16]:
clf_CV = DecisionTreeClassifier(random_state = 1, class_weight='balanced')

In [17]:
#create a grid search object and fit it to the data

clf_CV = GridSearchCV(estimator=clf_CV, param_grid= param_grid, cv=7, scoring='f1', verbose=1, n_jobs=1)
clf_CV.fit(X_train, y_train)

Fitting 7 folds for each of 18 candidates, totalling 126 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 126 out of 126 | elapsed:    1.4s finished


GridSearchCV(cv=7,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 10, 15],
                         'max_features': [0.25, 0.5, 0.75]},
             scoring='f1', verbose=1)

In [18]:
### Identify the best params 
print(clf_CV.best_params_)
print(clf_CV.best_estimator_)


#Identify the best score during fitting with cross-validation
print(clf_CV.best_score_)


{'criterion': 'gini', 'max_depth': 5, 'max_features': 0.5}
DecisionTreeClassifier(class_weight='balanced', max_depth=5, max_features=0.5,
                       random_state=1)
0.5714285714285714


In [19]:
#predict on the test set
test_preds_clf_CV = clf_CV.best_estimator_.predict(X_test)

# checking accuracy and F1
print("Accuracy:",metrics.accuracy_score(y_test, test_preds_clf_CV))
print("F1:",metrics.f1_score(y_test, test_preds_clf_CV))


Accuracy: 0.9583333333333334
F1: 0.5


## Random Forest

In [20]:
# Instantiate the classifier using 100 trees
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 1, n_estimators=100, max_depth=7, max_features=10)

In [21]:
#fit the model to the training data
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, max_features=10, random_state=1)

In [22]:
#use the fitted model to predict on the test data
rfc_preds = rfc.predict(X_test)
rfc_train = rfc.predict(X_train)

rfc_f1 = metrics.f1_score(y_test, rfc_preds)
rfc_acc = metrics.accuracy_score(y_test, rfc_preds)
train = metrics.f1_score(y_train, rfc_train)

# checking accuracy on the test data
print('Test Accuracy score: ', rfc_acc)
print('Test F1 score: ', rfc_f1)


Test Accuracy score:  1.0
Test F1 score:  1.0


# Infielders

## Modeling

In [24]:
#import dataframe
df = pd.read_pickle('infielders-df_2020-08-19a.p')

In [25]:
df['AVG'] = df['H']/df['AB']
df['OBP'] = (df['H'] + df['BB'] + df['HBP'])/(df['AB']+df['BB'] + df['HBP'] + df['SH'])

In [26]:
df = df.drop(['bCS', 'SH', 'HBP', 'bCS_ps', 'SH_ps', 'HBP_ps', 'teamID'], axis=1)

In [27]:
df.head()

,playerID,bG,AB,R,H,2B,3B,HR,RBI,bSB,BB,SO,bG_ps,AB_ps,R_ps,H_ps,2B_ps,3B_ps,HR_ps,RBI_ps,bSB_ps,BB_ps,SO_ps,fG,PO,A,E,DP,fG_ps,PO_ps,A_ps,E_ps,DP_ps,nu_sns,lst_sn,induct,position,AVG,OBP
1,adairje01,1165,4019,378,1022,163,19,57,366.0,29.0,208,499.0,5.0,16.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0,3.0,1167,2380,2878,103.0,618,4.0,7.0,12.0,0.0,1.0,13,1970,0,infielder,0.254292,0.291015
2,adamsbo03,1281,4019,591,1082,188,49,37,303.0,67.0,414,447.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1066,1748,2262,158.0,371,0.0,0.0,0.0,0.0,0.0,14,1959,0,infielder,0.269221,0.334143
3,adcocjo01,1959,6606,823,1832,295,35,336,1122.0,20.0,594,1059.0,9.0,28.0,2.0,7.0,0.0,0.0,0.0,2.0,0.0,1.0,5.0,1811,13678,900,102.0,1234,8.0,61.0,4.0,1.0,2.0,17,1966,0,infielder,0.277324,0.336085
5,aguaylu01,568,1104,142,260,43,10,37,109.0,7.0,94,220.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,503,483,867,54.0,148,0.0,0.0,0.0,0.0,0.0,10,1989,0,infielder,0.235507,0.303571
7,alexama02,594,1271,166,293,50,12,15,115.0,37.0,86,276.0,5.0,5.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,534,499,858,53.0,161,1.0,2.0,2.0,0.0,0.0,11,2006,0,infielder,0.230527,0.279103


In [28]:
df['induct'].value_counts()

0    590
1     43
Name: induct, dtype: int64

In [29]:
df.shape

(633, 39)

In [30]:
# Create matrix of features
X = df.drop(['induct', 'position', 'playerID'], axis = 1) 

# Create target variable
y = df['induct']

feature_cols = X.columns

In [31]:
X.shape

(633, 36)

In [32]:
# Use x and y variables to split the training data into train and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,test_size=0.20)

## Logistic Regression

### W/ Class Imbalance

In [33]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(solver='liblinear')

lr_clf.fit(X_train, y_train)

y_pred_test = lr_clf.predict(X_test)
y_pred_train = lr_clf.predict(X_train)


# checking accuracy
print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_test))


# checking accuracy
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_test))
print('Train F1 Score: ', metrics.f1_score(y_train, y_pred_train))

Test Accuracy score:  0.9291338582677166
Test F1 score:  0.5263157894736842
Train F1 Score:  0.90625


## Decision Tree

In [34]:
clf = DecisionTreeClassifier()


In [35]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#predict the training set
y_pred_train = clf.predict(X_train)

#Predict the response for test dataset
y_pred_test = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Testing Accuracy Score:",metrics.accuracy_score(y_test, y_pred_test))
print("Testing F1 Score:",metrics.f1_score(y_test, y_pred_test))

Testing Accuracy Score: 0.937007874015748
Testing F1 Score: 0.5


### GridsearchCV

In [36]:
from sklearn.model_selection import GridSearchCV

In [37]:
#create a dictionary of all the parameters you want to tune
param_grid = { 
    'max_features': [.25, .5, .75],
    'max_depth' : [5, 10, 15],
    'criterion' : ['gini','entropy']}

In [38]:
clf_CV = DecisionTreeClassifier(random_state = 1, class_weight='balanced')

In [39]:
#create a grid search object and fit it to the data

clf_CV = GridSearchCV(estimator=clf_CV, param_grid= param_grid, cv=7, scoring='f1', verbose=1, n_jobs=1)
clf_CV.fit(X_train, y_train)

Fitting 7 folds for each of 18 candidates, totalling 126 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 126 out of 126 | elapsed:    1.3s finished


GridSearchCV(cv=7,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 10, 15],
                         'max_features': [0.25, 0.5, 0.75]},
             scoring='f1', verbose=1)

In [40]:
### Identify the best params 
print(clf_CV.best_params_)
print(clf_CV.best_estimator_)


#Identify the best score during fitting with cross-validation
print(clf_CV.best_score_)


{'criterion': 'entropy', 'max_depth': 5, 'max_features': 0.25}
DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=5, max_features=0.25, random_state=1)
0.5300889586603872


In [41]:
#predict on the test set
test_preds_clf_CV = clf_CV.best_estimator_.predict(X_test)

# checking accuracy and F1
print("Accuracy:",metrics.accuracy_score(y_test, test_preds_clf_CV))
print("F1:",metrics.f1_score(y_test, test_preds_clf_CV))


Accuracy: 0.905511811023622
F1: 0.5


## Random Forest

In [42]:
# Instantiate the classifier using 100 trees
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 1, n_estimators=100, max_depth=7, max_features=10)

In [43]:
#fit the model to the training data
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, max_features=10, random_state=1)

In [44]:
#use the fitted model to predict on the test data
rfc_preds = rfc.predict(X_test)
rfc_train = rfc.predict(X_train)

rfc_f1 = metrics.f1_score(y_test, rfc_preds)
rfc_acc = metrics.accuracy_score(y_test, rfc_preds)
train = metrics.f1_score(y_train, rfc_train)

# checking accuracy on the test data
print('Test Accuracy score: ', rfc_acc)
print('Test F1 score: ', rfc_f1)


Test Accuracy score:  0.9606299212598425
Test F1 score:  0.6666666666666666


# Outfielders

## Modeling

In [45]:
#import dataframe
df = pd.read_pickle('outfielders-df_2020-08-19a.p')

In [46]:
df['AVG'] = df['H']/df['AB']
df['OBP'] = (df['H'] + df['BB'] + df['HBP'])/(df['AB']+df['BB'] + df['HBP'] + df['SH'])

In [47]:
df = df.drop(['bCS', 'SH', 'HBP', 'bCS_ps', 'SH_ps', 'HBP_ps', 'teamID'], axis=1)

In [48]:
df.head()

,playerID,bG,AB,R,H,2B,3B,HR,RBI,bSB,BB,SO,bG_ps,AB_ps,R_ps,H_ps,2B_ps,3B_ps,HR_ps,RBI_ps,bSB_ps,BB_ps,SO_ps,fG,PO,A,E,DP,fG_ps,PO_ps,A_ps,E_ps,DP_ps,nu_sns,lst_sn,induct,position,AVG,OBP
0,aaronha01,3298,12364,2174,3771,624,98,755,2297.0,240.0,1402,1383.0,17.0,69.0,11.0,25.0,4.0,1.0,6.0,16.0,0.0,5.0,13.0,3020,7436,429,144.0,218,18.0,31.0,1.0,0.0,0.0,23,1976,1,outfielder,0.304998,0.376655
4,ageeto01,1129,3912,558,999,170,27,130,433.0,167.0,342,918.0,8.0,32.0,5.0,8.0,1.0,0.0,3.0,5.0,3.0,4.0,10.0,1073,2371,53,61.0,18,8.0,28.0,0.0,0.0,0.0,12,1973,0,outfielder,0.255368,0.319100
6,aldremi01,930,2147,277,565,104,9,41,271.0,19.0,314,381.0,8.0,11.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,696,2388,159,19.0,187,4.0,5.0,0.0,0.0,0.0,10,1996,0,outfielder,0.263158,0.356452
13,allisbo01,1541,5032,811,1281,216,53,256,796.0,84.0,795,1033.0,10.0,26.0,3.0,2.0,1.0,0.0,1.0,3.0,1.0,3.0,10.0,1465,3546,162,84.0,108,7.0,17.0,0.0,0.0,0.0,13,1970,0,outfielder,0.254571,0.358478
18,aloufe01,2082,7339,985,2101,359,49,206,852.0,107.0,423,706.0,8.0,27.0,2.0,7.0,1.0,1.0,0.0,1.0,0.0,1.0,4.0,2006,6537,322,96.0,330,7.0,9.0,0.0,1.0,0.0,17,1974,0,outfielder,0.286279,0.328706


In [49]:
df['induct'].value_counts()

0    459
1     30
Name: induct, dtype: int64

In [50]:
df.shape

(489, 39)

In [51]:
# Create matrix of features
X = df.drop(['induct', 'position', 'playerID'], axis = 1) 

# Create target variable
y = df['induct']

feature_cols = X.columns

In [52]:
X.shape

(489, 36)

In [53]:
# Use x and y variables to split the training data into train and test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1,test_size=0.20)

## Logistic Regression

### W/ Class Imbalance

In [54]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(solver='liblinear')

lr_clf.fit(X_train, y_train)

y_pred_test = lr_clf.predict(X_test)
y_pred_train = lr_clf.predict(X_train)


# checking accuracy
print('Test Accuracy score: ', metrics.accuracy_score(y_test, y_pred_test))


# checking accuracy
print('Test F1 score: ', metrics.f1_score(y_test, y_pred_test))
print('Train F1 Score: ', metrics.f1_score(y_train, y_pred_train))

Test Accuracy score:  0.9489795918367347
Test F1 score:  0.28571428571428575
Train F1 Score:  1.0


## Decision Tree

In [55]:
clf = DecisionTreeClassifier()


In [56]:
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#predict the training set
y_pred_train = clf.predict(X_train)

#Predict the response for test dataset
y_pred_test = clf.predict(X_test)

# Model Accuracy, how often is the classifier correct?
print("Testing Accuracy Score:",metrics.accuracy_score(y_test, y_pred_test))
print("Testing F1 Score:",metrics.f1_score(y_test, y_pred_test))

Testing Accuracy Score: 0.9693877551020408
Testing F1 Score: 0.5714285714285715


### GridsearchCV

In [57]:
from sklearn.model_selection import GridSearchCV

In [58]:
#create a dictionary of all the parameters you want to tune
param_grid = { 
    'max_features': [.25, .5, .75],
    'max_depth' : [5, 10, 15],
    'criterion' : ['gini','entropy']}

In [59]:
clf_CV = DecisionTreeClassifier(random_state = 1, class_weight='balanced')

In [60]:
#create a grid search object and fit it to the data

clf_CV = GridSearchCV(estimator=clf_CV, param_grid= param_grid, cv=7, scoring='f1', verbose=1, n_jobs=1)
clf_CV.fit(X_train, y_train)

Fitting 7 folds for each of 18 candidates, totalling 126 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 126 out of 126 | elapsed:    1.2s finished


GridSearchCV(cv=7,
             estimator=DecisionTreeClassifier(class_weight='balanced',
                                              random_state=1),
             n_jobs=1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 10, 15],
                         'max_features': [0.25, 0.5, 0.75]},
             scoring='f1', verbose=1)

In [61]:
### Identify the best params 
print(clf_CV.best_params_)
print(clf_CV.best_estimator_)


#Identify the best score during fitting with cross-validation
print(clf_CV.best_score_)


{'criterion': 'entropy', 'max_depth': 5, 'max_features': 0.25}
DecisionTreeClassifier(class_weight='balanced', criterion='entropy',
                       max_depth=5, max_features=0.25, random_state=1)
0.648886827458256


In [62]:
#predict on the test set
test_preds_clf_CV = clf_CV.best_estimator_.predict(X_test)

# checking accuracy and F1
print("Accuracy:",metrics.accuracy_score(y_test, test_preds_clf_CV))
print("F1:",metrics.f1_score(y_test, test_preds_clf_CV))


Accuracy: 0.9591836734693877
F1: 0.5


## Random Forest

In [63]:
# Instantiate the classifier using 100 trees
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state = 1, n_estimators=100, max_depth=7, max_features=10)

In [64]:
#fit the model to the training data
rfc.fit(X_train, y_train)

RandomForestClassifier(max_depth=7, max_features=10, random_state=1)

In [65]:
#use the fitted model to predict on the test data
rfc_preds = rfc.predict(X_test)
rfc_train = rfc.predict(X_train)

rfc_f1 = metrics.f1_score(y_test, rfc_preds)
rfc_acc = metrics.accuracy_score(y_test, rfc_preds)
train = metrics.f1_score(y_train, rfc_train)

# checking accuracy on the test data
print('Test Accuracy score: ', rfc_acc)
print('Test F1 score: ', rfc_f1)


Test Accuracy score:  0.9897959183673469
Test F1 score:  0.8
